In [2]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import json 


In [3]:
base_path = 'E:\\indoor-location-navigation\\'

In [4]:
# pull out all the buildings actually used in the test set, given current method we don't need the other ones
ssubm = pd.read_csv(base_path + 'sample_submission.csv')

# only 24 of the total buildings are used in the test set, 
# this allows us to greatly reduce the intial size of the dataset

ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
used_buildings = sorted(ssubm_df[0].value_counts().index.tolist())

# dictionary used to map the floor codes to the values used in the submission file. 
floor_map = {"B2":-2, "B1":-1, "F1":0, "F2": 1, "F3":2, "F4":3, "F5":4, "F6":5, "F7":6,"F8":7, "F9":8,
             "1F":0, "2F":1, "3F":2, "4F":3, "5F":4, "6F":5, "7F":6, "8F": 7, "9F":8}

In [5]:
# get only the wifi bssid that occur over 1000 times(this number can be experimented with)
# these will be the only ones used when constructing features
bssid = dict()

In [6]:


for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train\\'+building+'\\*')))
    print(building)
    wifi = list()
    for folder in folders:
        print(folder)
        floor = floor_map[folder.split('\\')[-1]]
        print(floor)
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            with open(file, encoding="utf8") as f:
                txt = f.readlines()
                for e, line in enumerate(txt):
                    tmp = line.strip().split()
                    if tmp[1] == "TYPE_WIFI":
                        wifi.append(tmp)
    df = pd.DataFrame(wifi)
    #top_bssid = df[3].value_counts().iloc[:500].index.tolist()
    value_counts = df[3].value_counts()
    top_bssid = value_counts[value_counts > 500].index.tolist()
    print(len(top_bssid))
    bssid[building] = top_bssid
    del df
    del wifi
    gc.collect()

5a0546857ecc773753327266
E:\indoor-location-navigation\train\5a0546857ecc773753327266\B1
-1
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F1
0
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F2
1
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F3
2
E:\indoor-location-navigation\train\5a0546857ecc773753327266\F4
3
1651
5c3c44b80379370013e0fd2b
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\B1
-1
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F1
0
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F2
1
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F3
2
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F4
3
E:\indoor-location-navigation\train\5c3c44b80379370013e0fd2b\F5
4
1340
5d27075f03f801723c2e360f
E:\indoor-location-navigation\train\5d27075f03f801723c2e360f\B1
-1
E:\indoor-location-navigation\train\5d27075f03f801723c2e360f\F1
0
E:\indoor-location-navigation\train\5d27075f03f801723c

1768
5da958dd46f8266d0737457b
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\B1
-1
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F1
0
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F2
1
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F3
2
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F4
3
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F5
4
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F6
5
E:\indoor-location-navigation\train\5da958dd46f8266d0737457b\F7
6
1580
5dbc1d84c1eb61796cf7c010
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\B1
-1
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F2
1
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F3
2
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F4
3
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F5
4
E:\indoor-location-navigation\train\5dbc1d84c1eb61796cf7c010\F6
5
E:\indoor-loca

In [8]:
with open("bssid_1000.json", "w") as f:
    json.dump(bssid, f)

In [21]:
with open("bssid_1000.json") as f:
    bssid = json.load(f)

In [9]:
# generate all the training data 
building_dfs = dict()

for building in used_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train/', building +'/*')))
    dfs = list()
    index = sorted(bssid[building])
    print(building)
    for folder in folders:
        floor = floor_map[folder.split('\\')[-1]]
        files = glob.glob(os.path.join(folder, "*.txt"))
        for file in files:
            wifi = list()
            waypoint = list()
            with open(file, encoding="utf8") as f:
                txt = f.readlines()
            for line in txt:
                line = line.strip().split()
                if line[1] == "TYPE_WAYPOINT":
                    waypoint.append(line)
                if line[1] == "TYPE_WIFI":
                    wifi.append(line)

            df = pd.DataFrame(np.array(wifi))    

            # generate a feature, and label for each wifi block
            for gid, g in df.groupby(0):
                dists = list()
                for e, k in enumerate(waypoint):
                    dist = abs(int(gid) - int(k[0]))
                    dists.append(dist)
                nearest_wp_index = np.argmin(dists)
                
                # calculating x and y for wifi block
                calculated_coords = True
                
                if len(dists)>1:
                    second_nearest_wp_index = np.argsort(dists)[1]
                else:
                    calculated_coords = False
                    
                # need to define which time is in future
                if calculated_coords and int(waypoint[nearest_wp_index][0])> int(waypoint[second_nearest_wp_index][0]):
                    timestamp_future = int(waypoint[nearest_wp_index][0])
                    timestamp_past = int(waypoint[second_nearest_wp_index][0])
                    x_future = float(waypoint[nearest_wp_index][2])
                    y_future = float(waypoint[nearest_wp_index][3])
                    x_past = float(waypoint[second_nearest_wp_index][2])
                    y_past = float(waypoint[second_nearest_wp_index][3])
                elif calculated_coords:
                    timestamp_past = int(waypoint[nearest_wp_index][0])
                    timestamp_future = int(waypoint[second_nearest_wp_index][0])
                    x_past = float(waypoint[nearest_wp_index][2])
                    y_past = float(waypoint[nearest_wp_index][3])
                    x_future = float(waypoint[second_nearest_wp_index][2])
                    y_future = float(waypoint[second_nearest_wp_index][3])
                
                
                if calculated_coords and timestamp_past < int(gid) < timestamp_future: #Timestamp of wifi block between calculated timestamps
                    time_period =  timestamp_future - timestamp_past
                    x_inc_per_ms = (x_future - x_past) / time_period
                    y_inc_per_ms = (y_future - y_past) / time_period
                    x_calc = x_past + (int(gid)-timestamp_past)/time_period * x_inc_per_ms 
                    y_calc = y_past + (int(gid)-timestamp_past)/time_period * y_inc_per_ms
                    #print('Calculated coords are: ', x_calc, y_calc, 'floor ', floor, 'path ', file.split('\\')[-1].split('.')[0])
                    #print('Timestamps are: ', timestamp_past, int(gid), timestamp_future )
                else:
                    calculated_coords = False
                    # print("Calculated coords = False")
                    # print("Timestamps are: ", timestamp_past, int(gid), timestamp_future )
                    
                    
                g = g.drop_duplicates(subset=3)
                tmp = g.iloc[:,3:5]
                feat = tmp.set_index(3).reindex(index).replace(np.nan, -999).T
                feat["x"] = x_calc if calculated_coords else float(waypoint[nearest_wp_index][2])
                feat["y"] = y_calc if calculated_coords else float(waypoint[nearest_wp_index][3])
                feat["f"] = floor
                feat["path"] = file.split('\\')[-1].split('.')[0] # useful for crossvalidation
                dfs.append(feat)
                
    building_df = pd.concat(dfs)
    building_dfs[building] = df
    building_df.to_csv(building+"_1000_train.csv")

5a0546857ecc773753327266
5c3c44b80379370013e0fd2b
5d27075f03f801723c2e360f
5d27096c03f801723c31e5e0
5d27097f03f801723c320d97
5d27099f03f801723c32511d
5d2709a003f801723c3251bf
5d2709b303f801723c327472
5d2709bb03f801723c32852c
5d2709c303f801723c3299ee
5d2709d403f801723c32bd39
5d2709e003f801723c32d896
5da138274db8ce0c98bbd3d2
5da1382d4db8ce0c98bbe92e
5da138314db8ce0c98bbf3a0
5da138364db8ce0c98bc00f1
5da1383b4db8ce0c98bc11ab
5da138754db8ce0c98bca82f
5da138764db8ce0c98bcaa46
5da1389e4db8ce0c98bd0547
5da138b74db8ce0c98bd4774
5da958dd46f8266d0737457b
5dbc1d84c1eb61796cf7c010
5dc8cea7659e181adb076a3f


In [10]:
ssubm_building_g = ssubm_df.groupby(0) #Grouped by building
feature_dict = dict()

for gid0, g0 in ssubm_building_g:
    index = sorted(bssid[g0.iloc[0,0]])
    feats = list()
#     print(gid0, 'gid0')
#     print(g0, 'g0')
#     print()
    for gid,g in g0.groupby(1):
        # get all wifi time locations, 
#         print(gid, 'gid')
#         print(g, 'g')
#         print()
        with open(os.path.join(base_path, 'test\\' + g.iloc[0,1] + '.txt'), encoding="utf8") as f:
            txt = f.readlines()

        wifi = list()

        for line in txt:
            line = line.strip().split()
            if line[1] == "TYPE_WIFI":
                wifi.append(line)

        wifi_df = pd.DataFrame(wifi) #All wifi points for certain path
        wifi_points = pd.DataFrame(wifi_df.groupby(0).count().index.tolist()) #Timestamps for wifi for certain path
        #print(wifi_points)
        for timepoint in g.iloc[:,2].tolist():
            deltas = (wifi_points.astype(int) - int(timepoint)).abs()
            min_delta_idx = deltas.values.argmin()
            wifi_block_timestamp = wifi_points.iloc[min_delta_idx].values[0]
            
            wifi_block = wifi_df[wifi_df[0] == wifi_block_timestamp].drop_duplicates(subset=3)
            feat = wifi_block.set_index(3)[4].reindex(index).fillna(-999)

            feat['site_path_timestamp'] = g.iloc[0,0] + "_" + g.iloc[0,1] + "_" + timepoint
            feats.append(feat)
    feature_df = pd.concat(feats, axis=1).T
    feature_df.to_csv(gid0+"_1000_test.csv")
    feature_dict[gid0] = feature_df

In [11]:
import xgboost as xgb

In [12]:
feature_path = 'C:\\Users\\user\\Documents\\LightHouseLabs\\W09\\'

In [13]:
train_files = sorted(glob.glob(os.path.join(feature_path, '*_train.csv')))
test_files = sorted(glob.glob(os.path.join(feature_path, '*_test.csv')))

In [20]:
predictions = list()

for e, file in enumerate(train_files):
    data = pd.read_csv(file, index_col=0)
    test_data = pd.read_csv(test_files[e], index_col=0)

    # simple grid search for tuning using path and groupkfold cv
    #for i in [127]:
    #    for n in [75]:
    #        modely = lgb.LGBMRegressor(
    #        n_estimators=n, num_leaves=i, n_jobs=1)
    #        gkf = model_selection.GroupKFold()
    #        scores = model_selection.cross_val_score(
    #            modely, x_train, y_trainy, scoring='neg_mean_squared_error',
    #            cv=gkf, groups=data.iloc[:,-1], n_jobs=5)
    #        print(i, n, l, scores, scores.mean())
    
    data = data.sample(frac=1)
    x_train = data.iloc[:int(len(data)*0.75),:-4]
    y_trainy = data.iloc[:int(len(data)*0.75),-3]
    y_trainx = data.iloc[:int(len(data)*0.75),-4]
    y_trainf = data.iloc[:int(len(data)*0.75),-2] + 2 #To make floors classes from 0 to 10
    
    x_test = data.iloc[int(len(data)*0.75):,:-4]
    y_testy = data.iloc[int(len(data)*0.75):,-3]
    y_testx = data.iloc[int(len(data)*0.75):,-4]
    y_testf = data.iloc[int(len(data)*0.75):,-2] + 2 #To make floors classes from 0 to 10
    
    
    dtrainX = xgb.DMatrix(x_train, label=y_trainx)
    dtestX = xgb.DMatrix(x_test, label=y_testx)
    
    dtrainY = xgb.DMatrix(x_train, label=y_trainy)
    dtestY = xgb.DMatrix(x_test, label=y_testy)
    
    dtrainF = xgb.DMatrix(x_train, label=y_trainf)
    dtestF = xgb.DMatrix(x_test, label=y_testf)
    
    paramsXY = {
    # Parameters that we are going to tune.
    'max_depth':25,
    'min_child_weight': 0.5,
    'eta':.2,
    'subsample': 1,
    'colsample_bytree': 0.5,
    # Other parameters
    'objective':'reg:squarederror',
    'eval_metric':'rmse',
    'booster':'gbtree',
    'random_state':17,
    'verbosity':1,
    'n_jobs':-1
    }
    
    paramsF = {
    # Parameters that we are going to tune.
    'max_depth':50,
    'min_child_weight': 0.5,
    'eta':.2,
    'subsample': 1,
    'colsample_bytree': 0.5,
    # Other parameters
    'objective':'multi:softmax',
    'eval_metric':'merror',
    'num_class':11,
    'booster':'gbtree',
    'random_state':17,
    'verbosity':1,
    'n_jobs':-1
    }
    
    num_boost_round=999
    
    xgb_rX = xgb.train(
    paramsXY,
    dtrainX,
    num_boost_round=num_boost_round,
    evals=[(dtestX, "Test")],
    early_stopping_rounds=50
    )
    
    xgb_rY = xgb.train(
    paramsXY,
    dtrainY,
    num_boost_round=num_boost_round,
    evals=[(dtestY, "Test")],
    early_stopping_rounds=50
    )
    
    xgb_cF = xgb.train(
    paramsF,
    dtrainF,
    num_boost_round=num_boost_round,
    evals=[(dtestF, "Test")],
    early_stopping_rounds=50
    )
    
#     modely = lgb.LGBMRegressor(
#         n_estimators=125, num_leaves=90)
#     modely.fit(x_train, y_trainy)

#     modelx = lgb.LGBMRegressor(
#         n_estimators=125, num_leaves=90)
#     modelx.fit(x_train, y_trainx)

#     modelf = lgb.LGBMClassifier(
#         n_estimators=125, num_leaves=90)
#     modelf.fit(x_train, y_trainf)
    
    test_predsx = xgb_rX.predict(xgb.DMatrix(test_data.iloc[:,:-1]))
    test_predsy = xgb_rY.predict(xgb.DMatrix(test_data.iloc[:,:-1]))
    test_predsf = xgb_cF.predict(xgb.DMatrix(test_data.iloc[:,:-1]+2))
    
    test_preds = pd.DataFrame(np.stack((test_predsf, test_predsx, test_predsy))).T
    test_preds.columns = ssubm.columns[1:]
    test_preds.index = test_data["site_path_timestamp"]
    test_preds["floor"] = test_preds["floor"].astype(int)-2 #Returning floor classes back to original
    predictions.append(test_preds)

[0]	Test-rmse:91.47842
[1]	Test-rmse:73.47754
[2]	Test-rmse:59.19684
[3]	Test-rmse:47.70837
[4]	Test-rmse:38.53964
[5]	Test-rmse:31.24737
[6]	Test-rmse:25.44468
[7]	Test-rmse:20.81526
[8]	Test-rmse:17.15256
[9]	Test-rmse:14.30425
[10]	Test-rmse:12.07721
[11]	Test-rmse:10.35408
[12]	Test-rmse:9.04813
[13]	Test-rmse:8.06333
[14]	Test-rmse:7.34137
[15]	Test-rmse:6.81959
[16]	Test-rmse:6.43027
[17]	Test-rmse:6.15161
[18]	Test-rmse:5.94840
[19]	Test-rmse:5.79796
[20]	Test-rmse:5.69568
[21]	Test-rmse:5.61480
[22]	Test-rmse:5.56354
[23]	Test-rmse:5.52180
[24]	Test-rmse:5.49491
[25]	Test-rmse:5.46731
[26]	Test-rmse:5.44425
[27]	Test-rmse:5.42501
[28]	Test-rmse:5.41174
[29]	Test-rmse:5.39944
[30]	Test-rmse:5.39358
[31]	Test-rmse:5.38475
[32]	Test-rmse:5.37700
[33]	Test-rmse:5.37308
[34]	Test-rmse:5.37117
[35]	Test-rmse:5.36772
[36]	Test-rmse:5.36548
[37]	Test-rmse:5.36130
[38]	Test-rmse:5.35823
[39]	Test-rmse:5.35723
[40]	Test-rmse:5.35580
[41]	Test-rmse:5.35510
[42]	Test-rmse:5.35406
[43]	Test

[18]	Test-rmse:3.59434
[19]	Test-rmse:3.42226
[20]	Test-rmse:3.31017
[21]	Test-rmse:3.22664
[22]	Test-rmse:3.16088
[23]	Test-rmse:3.11757
[24]	Test-rmse:3.08660
[25]	Test-rmse:3.06459
[26]	Test-rmse:3.04962
[27]	Test-rmse:3.03785
[28]	Test-rmse:3.02940
[29]	Test-rmse:3.02283
[30]	Test-rmse:3.01845
[31]	Test-rmse:3.01478
[32]	Test-rmse:3.01205
[33]	Test-rmse:3.00973
[34]	Test-rmse:3.00724
[35]	Test-rmse:3.00540
[36]	Test-rmse:3.00459
[37]	Test-rmse:3.00395
[38]	Test-rmse:3.00290
[39]	Test-rmse:3.00255
[40]	Test-rmse:3.00237
[41]	Test-rmse:3.00131
[42]	Test-rmse:3.00075
[43]	Test-rmse:2.99994
[44]	Test-rmse:2.99918
[45]	Test-rmse:2.99866
[46]	Test-rmse:2.99844
[47]	Test-rmse:2.99843
[48]	Test-rmse:2.99808
[49]	Test-rmse:2.99790
[50]	Test-rmse:2.99776
[51]	Test-rmse:2.99756
[52]	Test-rmse:2.99757
[53]	Test-rmse:2.99737
[54]	Test-rmse:2.99731
[55]	Test-rmse:2.99677
[56]	Test-rmse:2.99673
[57]	Test-rmse:2.99666
[58]	Test-rmse:2.99673
[59]	Test-rmse:2.99656
[60]	Test-rmse:2.99650
[61]	Test-r

[21]	Test-rmse:6.20640
[22]	Test-rmse:6.05325
[23]	Test-rmse:5.93831
[24]	Test-rmse:5.84789
[25]	Test-rmse:5.78314
[26]	Test-rmse:5.74005
[27]	Test-rmse:5.70599
[28]	Test-rmse:5.68371
[29]	Test-rmse:5.66317
[30]	Test-rmse:5.64792
[31]	Test-rmse:5.63543
[32]	Test-rmse:5.62606
[33]	Test-rmse:5.61746
[34]	Test-rmse:5.61124
[35]	Test-rmse:5.60748
[36]	Test-rmse:5.60215
[37]	Test-rmse:5.59850
[38]	Test-rmse:5.59480
[39]	Test-rmse:5.59394
[40]	Test-rmse:5.59269
[41]	Test-rmse:5.59111
[42]	Test-rmse:5.58898
[43]	Test-rmse:5.58769
[44]	Test-rmse:5.58671
[45]	Test-rmse:5.58629
[46]	Test-rmse:5.58500
[47]	Test-rmse:5.58354
[48]	Test-rmse:5.58194
[49]	Test-rmse:5.58085
[50]	Test-rmse:5.57957
[51]	Test-rmse:5.57901
[52]	Test-rmse:5.57939
[53]	Test-rmse:5.57903
[54]	Test-rmse:5.57854
[55]	Test-rmse:5.57846
[56]	Test-rmse:5.57731
[57]	Test-rmse:5.57706
[58]	Test-rmse:5.57646
[59]	Test-rmse:5.57617
[60]	Test-rmse:5.57592
[61]	Test-rmse:5.57565
[62]	Test-rmse:5.57522
[63]	Test-rmse:5.57493
[64]	Test-r

[366]	Test-rmse:5.56590
[367]	Test-rmse:5.56590
[368]	Test-rmse:5.56590
[369]	Test-rmse:5.56590
[370]	Test-rmse:5.56590
[371]	Test-rmse:5.56590
[372]	Test-rmse:5.56590
[373]	Test-rmse:5.56590
[374]	Test-rmse:5.56590
[375]	Test-rmse:5.56590
[376]	Test-rmse:5.56590
[377]	Test-rmse:5.56590
[378]	Test-rmse:5.56590
[379]	Test-rmse:5.56590
[380]	Test-rmse:5.56590
[381]	Test-rmse:5.56590
[382]	Test-rmse:5.56590
[383]	Test-rmse:5.56590
[384]	Test-rmse:5.56590
[385]	Test-rmse:5.56590
[0]	Test-rmse:84.90942
[1]	Test-rmse:68.20689
[2]	Test-rmse:54.83289
[3]	Test-rmse:44.09587
[4]	Test-rmse:35.62053
[5]	Test-rmse:28.84288
[6]	Test-rmse:23.43396
[7]	Test-rmse:19.12271
[8]	Test-rmse:15.69765
[9]	Test-rmse:13.01305
[10]	Test-rmse:10.88238
[11]	Test-rmse:9.24343
[12]	Test-rmse:8.00585
[13]	Test-rmse:7.05130
[14]	Test-rmse:6.33088
[15]	Test-rmse:5.79437
[16]	Test-rmse:5.40052
[17]	Test-rmse:5.12150
[18]	Test-rmse:4.92508
[19]	Test-rmse:4.78027
[20]	Test-rmse:4.68514
[21]	Test-rmse:4.62110
[22]	Test-rms

[326]	Test-rmse:4.41348
[327]	Test-rmse:4.41348
[328]	Test-rmse:4.41348
[329]	Test-rmse:4.41348
[330]	Test-rmse:4.41348
[331]	Test-rmse:4.41348
[332]	Test-rmse:4.41348
[333]	Test-rmse:4.41348
[0]	Test-merror:0.01355
[1]	Test-merror:0.00370
[2]	Test-merror:0.00041
[3]	Test-merror:0.00041
[4]	Test-merror:0.00041
[5]	Test-merror:0.00000
[6]	Test-merror:0.00000
[7]	Test-merror:0.00000
[8]	Test-merror:0.00000
[9]	Test-merror:0.00000
[10]	Test-merror:0.00000
[11]	Test-merror:0.00000
[12]	Test-merror:0.00000
[13]	Test-merror:0.00000
[14]	Test-merror:0.00000
[15]	Test-merror:0.00000
[16]	Test-merror:0.00000
[17]	Test-merror:0.00000
[18]	Test-merror:0.00000
[19]	Test-merror:0.00000
[20]	Test-merror:0.00000
[21]	Test-merror:0.00000
[22]	Test-merror:0.00000
[23]	Test-merror:0.00000
[24]	Test-merror:0.00000
[25]	Test-merror:0.00000
[26]	Test-merror:0.00000
[27]	Test-merror:0.00000
[28]	Test-merror:0.00000
[29]	Test-merror:0.00000
[30]	Test-merror:0.00000
[31]	Test-merror:0.00000
[32]	Test-merror:0

...

[63]	Test-rmse:4.32230
[64]	Test-rmse:4.32185
[65]	Test-rmse:4.32116
[66]	Test-rmse:4.32059
[67]	Test-rmse:4.32041
[68]	Test-rmse:4.32012
[69]	Test-rmse:4.31928
[70]	Test-rmse:4.31842
[71]	Test-rmse:4.31724
[72]	Test-rmse:4.31698
[73]	Test-rmse:4.31704
[74]	Test-rmse:4.31677
[75]	Test-rmse:4.31628
[76]	Test-rmse:4.31591
[77]	Test-rmse:4.31559
[78]	Test-rmse:4.31517
[79]	Test-rmse:4.31435
[80]	Test-rmse:4.31374
[81]	Test-rmse:4.31335
[82]	Test-rmse:4.31241
[83]	Test-rmse:4.31182
[84]	Test-rmse:4.31103
[85]	Test-rmse:4.31041
[86]	Test-rmse:4.31006
[87]	Test-rmse:4.30991
[88]	Test-rmse:4.30978
[89]	Test-rmse:4.30939
[90]	Test-rmse:4.30913
[91]	Test-rmse:4.30854
[92]	Test-rmse:4.30829
[93]	Test-rmse:4.30790
[94]	Test-rmse:4.30789
[95]	Test-rmse:4.30709
[96]	Test-rmse:4.30676
[97]	Test-rmse:4.30666
[98]	Test-rmse:4.30650
[99]	Test-rmse:4.30629
[100]	Test-rmse:4.30576
[101]	Test-rmse:4.30561
[102]	Test-rmse:4.30555
[103]	Test-rmse:4.30545
[104]	Test-rmse:4.30518
[105]	Test-rmse:4.30447
[106]

[406]	Test-rmse:4.29329
[407]	Test-rmse:4.29329
[408]	Test-rmse:4.29329
[409]	Test-rmse:4.29329
[410]	Test-rmse:4.29329
[411]	Test-rmse:4.29328
[412]	Test-rmse:4.29329
[413]	Test-rmse:4.29329
[414]	Test-rmse:4.29328
[415]	Test-rmse:4.29328
[416]	Test-rmse:4.29328
[417]	Test-rmse:4.29328
[418]	Test-rmse:4.29328
[419]	Test-rmse:4.29328
[420]	Test-rmse:4.29328
[421]	Test-rmse:4.29328
[422]	Test-rmse:4.29328
[423]	Test-rmse:4.29328
[424]	Test-rmse:4.29328
[425]	Test-rmse:4.29328
[426]	Test-rmse:4.29328
[427]	Test-rmse:4.29328
[428]	Test-rmse:4.29327
[429]	Test-rmse:4.29327
[430]	Test-rmse:4.29327
[431]	Test-rmse:4.29328
[432]	Test-rmse:4.29327
[433]	Test-rmse:4.29328
[434]	Test-rmse:4.29327
[435]	Test-rmse:4.29327
[436]	Test-rmse:4.29327
[437]	Test-rmse:4.29327
[438]	Test-rmse:4.29327
[439]	Test-rmse:4.29327
[440]	Test-rmse:4.29327
[441]	Test-rmse:4.29327
[442]	Test-rmse:4.29327
[443]	Test-rmse:4.29327
[444]	Test-rmse:4.29327
[445]	Test-rmse:4.29327
[446]	Test-rmse:4.29327
[447]	Test-rmse:

[265]	Test-rmse:3.96754
[266]	Test-rmse:3.96753
[267]	Test-rmse:3.96754
[268]	Test-rmse:3.96754
[269]	Test-rmse:3.96755
[270]	Test-rmse:3.96752
[271]	Test-rmse:3.96751
[272]	Test-rmse:3.96752
[273]	Test-rmse:3.96750
[274]	Test-rmse:3.96750
[275]	Test-rmse:3.96749
[276]	Test-rmse:3.96745
[277]	Test-rmse:3.96743
[278]	Test-rmse:3.96742
[279]	Test-rmse:3.96742
[280]	Test-rmse:3.96742
[281]	Test-rmse:3.96742
[282]	Test-rmse:3.96741
[283]	Test-rmse:3.96740
[284]	Test-rmse:3.96738
[285]	Test-rmse:3.96738
[286]	Test-rmse:3.96738
[287]	Test-rmse:3.96739
[288]	Test-rmse:3.96738
[289]	Test-rmse:3.96738
[290]	Test-rmse:3.96737
[291]	Test-rmse:3.96737
[292]	Test-rmse:3.96738
[293]	Test-rmse:3.96737
[294]	Test-rmse:3.96736
[295]	Test-rmse:3.96735
[296]	Test-rmse:3.96735
[297]	Test-rmse:3.96734
[298]	Test-rmse:3.96733
[299]	Test-rmse:3.96732
[300]	Test-rmse:3.96730
[301]	Test-rmse:3.96730
[302]	Test-rmse:3.96728
[303]	Test-rmse:3.96728
[304]	Test-rmse:3.96729
[305]	Test-rmse:3.96729
[306]	Test-rmse:

[30]	Test-rmse:4.81860
[31]	Test-rmse:4.77711
[32]	Test-rmse:4.72436
[33]	Test-rmse:4.68966
[34]	Test-rmse:4.66755
[35]	Test-rmse:4.65168
[36]	Test-rmse:4.62633
[37]	Test-rmse:4.60603
[38]	Test-rmse:4.59544
[39]	Test-rmse:4.58675
[40]	Test-rmse:4.57666
[41]	Test-rmse:4.56768
[42]	Test-rmse:4.56028
[43]	Test-rmse:4.55338
[44]	Test-rmse:4.54112
[45]	Test-rmse:4.53695
[46]	Test-rmse:4.52881
[47]	Test-rmse:4.51863
[48]	Test-rmse:4.51257
[49]	Test-rmse:4.51063
[50]	Test-rmse:4.50653
[51]	Test-rmse:4.50580
[52]	Test-rmse:4.50311
[53]	Test-rmse:4.49831
[54]	Test-rmse:4.49585
[55]	Test-rmse:4.49369
[56]	Test-rmse:4.49370
[57]	Test-rmse:4.48442
[58]	Test-rmse:4.48318
[59]	Test-rmse:4.47912
[60]	Test-rmse:4.48154
[61]	Test-rmse:4.48059
[62]	Test-rmse:4.47965
[63]	Test-rmse:4.47840
[64]	Test-rmse:4.48106
[65]	Test-rmse:4.48119
[66]	Test-rmse:4.47809
[67]	Test-rmse:4.47503
[68]	Test-rmse:4.47291
[69]	Test-rmse:4.47551
[70]	Test-rmse:4.47666
[71]	Test-rmse:4.47888
[72]	Test-rmse:4.47873
[73]	Test-r

[261]	Test-rmse:4.04379
[262]	Test-rmse:4.04381
[263]	Test-rmse:4.04384
[264]	Test-rmse:4.04386
[265]	Test-rmse:4.04384
[266]	Test-rmse:4.04382
[267]	Test-rmse:4.04385
[268]	Test-rmse:4.04382
[269]	Test-rmse:4.04388
[270]	Test-rmse:4.04387
[271]	Test-rmse:4.04385
[272]	Test-rmse:4.04383
[273]	Test-rmse:4.04384
[274]	Test-rmse:4.04383
[275]	Test-rmse:4.04383
[276]	Test-rmse:4.04387
[277]	Test-rmse:4.04388
[278]	Test-rmse:4.04387
[279]	Test-rmse:4.04390
[280]	Test-rmse:4.04391
[281]	Test-rmse:4.04389
[282]	Test-rmse:4.04393
[283]	Test-rmse:4.04393
[284]	Test-rmse:4.04391
[285]	Test-rmse:4.04392
[286]	Test-rmse:4.04397
[287]	Test-rmse:4.04401
[288]	Test-rmse:4.04403
[289]	Test-rmse:4.04399
[290]	Test-rmse:4.04401
[291]	Test-rmse:4.04403
[292]	Test-rmse:4.04403
[0]	Test-merror:0.02299
[1]	Test-merror:0.00766
[2]	Test-merror:0.00409
[3]	Test-merror:0.00230
[4]	Test-merror:0.00077
[5]	Test-merror:0.00051
[6]	Test-merror:0.00026
[7]	Test-merror:0.00026
[8]	Test-merror:0.00026
[9]	Test-merror:

In [21]:
# generate prediction file 
all_preds = pd.concat(predictions)


In [22]:
all_preds.reset_index(inplace=True)

In [23]:
all_preds = all_preds.reindex(ssubm.index)


In [24]:
all_preds.to_csv('submission.csv', index=False)

In [25]:
all_preds

,site_path_timestamp,floor,x,y
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,93.419945,94.215286
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,92.521774,98.937103
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,95.059166,109.164360
3,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,90.973976,108.964020
4,5a0546857ecc773753327266_046cfa46be49fc1083481...,0,92.254929,107.660622
...,...,...,...,...
10128,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,213.505310,90.501816
10129,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,212.864197,93.341698
10130,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,205.316360,105.719009
10131,5dc8cea7659e181adb076a3f_fd64de8c4a2fc5ebb0e9f...,5,205.844025,114.836006
